In [4]:
library(tidyverse)

source("../../evaluation_utils/plots_eda.R")

May take several hours to run. You can skip this step and go to the `evaluation` folder to see the results.

# Load and central correction

In [ ]:
path_to_data <- "./"

n_runs = 15

In [6]:
for(mode in c(
    "balanced",
    "mild_imbalanced", "strong_imbalanced"
)){
    # if folder does not exist, create it
    if(!dir.exists(paste0(path_to_data, mode, "/after/runs"))){
        dir.create(paste0(path_to_data, mode, "/after/runs"), recursive = T)
    }

    print(paste0("Processing mode: ", mode))

    metadata <- read.csv(paste0(path_to_data, mode, "/all_metadata.tsv"), sep = "\t") %>%
        as.data.frame()

    for(j in 1:n_runs){
        intensities <- read.csv(paste0(path_to_data, mode, "/before/intermediate/", j, "_intensities_data.tsv"), sep = "\t") %>%
            as.data.frame() %>%
            column_to_rownames("rowname")

        metadata <- metadata %>%
          mutate(condition = as.factor(condition), lab = as.factor(lab))

        design <- model.matrix(~ condition, metadata)
        colnames(design) <- c("Intercept", "condition")

        intensities_corrected <- limma::removeBatchEffect(
                intensities[,metadata$file],
                metadata$lab,
                design = design) %>%
            as.data.frame()

        # write to file
        write.table(intensities_corrected %>% rownames_to_column("rowname"),
                    paste0(path_to_data, mode, "/after/runs/", j, "_R_corrected.tsv"),
                    sep = "\t", row.names = F)

        print(paste0("\t\tSaved corrected intensities for mode: ", mode))
    }
}

[1] "Processing mode: balanced"
[1] "\t\tSaved corrected intensities for mode: balanced"
[1] "Processing mode: mild_imbalanced"
[1] "\t\tSaved corrected intensities for mode: mild_imbalanced"
[1] "Processing mode: strong_imbalanced"
[1] "\t\tSaved corrected intensities for mode: strong_imbalanced"


# Session info

In [7]:
print(sessionInfo())

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Linux Mint 22

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.12.0 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_DK.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] viridis_0.6.5     viridisLite_0.4.3 ggsci_4.2.0       umap_0.2.10.0    
 [5] patchwork_1.3.2   gridExtra_2.3     lubridate_1.9.3   forcats_1.0.0    
 [9] stringr_1.5.1     dplyr_1.1.4       purrr